In [ ]:
pip install unidecode scikit-learn BeautifulSoup4 pandas requests lxml selenium

In [ ]:
from sklearn.preprocessing import StandardScaler
from requests.auth import HTTPBasicAuth
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import numpy as np
import unidecode
import requests
import locale
import pickle
import re 
import lxml
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import regex as re

In [ ]:
search_dict_main_page = {
    'rating': {'name': 'div', 'attrs': {"data-testid": "Rating"}},
    'director': {'name': 'a', 'attrs': {"data-testid": "link"}},
    'duration': {'name': 'span', 'attrs': {"data-testid": "duration"}},
    'genres': {'name': 'span', 'attrs': {"data-testid": "genres"}},
    'ranking': {'name': 'span', 'attrs':{'data-testid': 'product-title-wrapper'}}
}

search_dict_film_page = {
    'film_year': {'name': 'p', 'attrs': {"class": "Text__SCTitle-sc-1aoldkr-1 CoverProductInfos__StyledText-sc-1un0kh1-13 eGhlHy jugtWW"}},
    'original_title':{'name': 'p', 'attrs': {"class": "Text__SCTitle-sc-1aoldkr-1 CoverProductInfos__StyledText-sc-1un0kh1-13 eGhlHy kuMSs"}}
}


In [ ]:
class RequestSensCritique:
    def __init__(self):
        self._base_urls = 'https://www.senscritique.com/'
    
    def create_soup_from_url(self, url_src):
        page_content = requests.get(url_src).content
        
        return BeautifulSoup(page_content, "html.parser")

    def find_text(self, element, search_criteria):
        found_element = element.find(**search_criteria)
        return found_element.text if found_element else None

    def get_full_page(self, link):
        full_url = self._base_urls + link
        driver = webdriver.Chrome()

        try:
            driver.get(full_url)

            for _ in range(8):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
            
            page_source = driver.page_source

        finally:
            driver.quit()

        self.create_soup_from_url(page_source)
        
    def get_film_page_info(self, film_link, dict_name):
        film_url = self._base_urls + link
        film_page_soup = self.create_soup_from_url(film_url)

        film_page_dict = {key: self.find_text(film, criteria) for key, criteria in dict_name.items()}

        stats_elements = film_page_soup.select('p.Text__SCText-sc-1aoldkr-0.Stats__Text-sc-1u6v943-2.gATBvI')
        if len(stats_elements) == 3:
            film_page_dict['stars'] = stats_elements[0].text
            film_page_dict['saves'] = stats_elements[1].text
            film_page_dict['favorites'] = stats_elements[2].text
        
        critique_number_text = film_page_soup.select('.NavigationTab__WrapperTextStyled-sc-18dtd9d-7')[2].text
        film_page_dict['critique_number'] = int(critique_number_text.split('(')[-1].split(')')[0])

        return film_page_dict
    
    def get_main_page(self, link, main_dict, film_info_dict):
        film_soup = self.get_full_page(link)

        films = film_soup.find_all('div', class_="ProductListItem__Wrapper-sc-1jkxxpj-1 kusRkg")

        all_films_dict = {}
        count = 0

        for film in films:
            if count <= 2:
                title = self.text(film, {'name': 'a', 'attrs': {"data-testid": "product-title"}})

                new_film_dict = {key: self.find_text(film, criteria) for key, criteria in main_dict.items()}

                film_link = film.find('a', {"data-testid": "product-title"})['href']
                new_film_dict.update(self.get_film_page_info(film_link, film_info_dict))
                
                all_films_dict[title] = new_film_dict
                count += 1
        
        return all_films_dict


In [ ]:
create_request = RequestSensCritique()

top_111_link = 'films/tops/top111'
create_request.get_main_page(top_111_link, search_dict_main_page,search_dict_film_page )

In [ ]:
pip install webdriver_manager


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)
